Salma Aulia <br>
211351134 <br>
Pagi A

# Import Dataset Kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

In [ ]:
!kaggle datasets download -d starbucks/starbucks-menu

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/starbucks-menu.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

# Import Library

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import sklearn.cluster as KMeans
from sklearn.cluster import KMeans

#Data Discovery

In [ ]:
df = pd.read_csv('/content/starbucks-menu-nutrition-drinks.csv')

In [ ]:
df.head()

#Data Preparation

In [ ]:
df.rename(columns = {"Unnamed: 0": "Menu"},inplace=True)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
y = [x for x in df['Calories'] if x.isnumeric() == False]
print(y)

In [ ]:
df['Calories'] = df['Calories'].str.replace('-','0')
df['Fat (g)'] = df['Fat (g)'].str.replace('-','0')
df['Carb. (g)'] = df['Carb. (g)'].str.replace('-','0')
df['Fiber (g)'] = df['Fiber (g)'].str.replace('-','0')
df['Protein'] = df['Protein'].str.replace('-','0')
df['Sodium'] = df['Sodium'].str.replace('-','0')

In [ ]:
df['Calories'] = pd.to_numeric(df['Calories'])
df['Fat (g)'] = pd.to_numeric(df['Fat (g)'])
df['Carb. (g)'] = pd.to_numeric(df['Carb. (g)'])
df['Fiber (g)'] = pd.to_numeric(df['Fiber (g)'])
df['Protein'] = pd.to_numeric(df['Protein'])
df['Sodium'] = pd.to_numeric(df['Sodium'])

In [ ]:
df.info()

In [ ]:
df.head()

#EDA

In [ ]:
g = sns.scatterplot(data=df, x="Calories", y="Sodium", hue="Menu")
#g.legend_.remove()
sns.move_legend(g, "upper left", bbox_to_anchor=(1, 1))


Dari grafik diatas bisa ita lihat bahwa Menu Cinnamon Dolce Frapppucino adalah menu yang memiliki kandungan kalori dan sodium terbesar

In [ ]:
g = sns.scatterplot(data=df, x="Calories", y="Carb. (g)", hue="Menu")
#g.legend_.remove()
sns.move_legend(g, "upper left", bbox_to_anchor=(1, 1))

Dari grafik diatas bisa ita lihat bahwa Menu Cafe Vanila Frapppucino adalah menu yang memiliki kandungan kalori dan sodium terbesar

In [ ]:
models = df.groupby('Menu').count()[['Calories']].sort_values(by='Calories',ascending=True).reset_index()
models = models.rename(columns={'Menu':'Menus'})

In [ ]:
sns.lineplot(data=df, x="Calories", y="Sodium", orient="y")


Grafik diatas adalah data perbandingan jumlah kalori dan sodium pada menu

In [ ]:
sns.lineplot(data=df, x="Calories", y="Carb. (g)", orient="y")

Grafik diatas adalah data perbandingan jumlah kalori dan Carbohidrat pada menu

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot(df['Calories'])

Grafik diatas adalah persebaran data dari kolom calories

#Modelling

In [ ]:
df.info()

In [ ]:
x = df.drop(['Menu','Fat (g)','Fiber (g)','Protein'], axis=1)

In [ ]:
print (x)

In [ ]:
clusters=[]
for i in range(1,11):
  kmeans = KMeans(n_clusters=i).fit(x)
  clusters.append(kmeans.inertia_)

fig,ax=plt.subplots(figsize=(12,8))
sns.lineplot(x=list(range(1,11)), y=clusters, ax=ax)
ax.set_title('mencari elbow')
ax.set_xlabel('clusters')
ax.set_xlabel('inertia')

In [ ]:
n_clust = 3
kmean = KMeans(n_clusters=n_clust).fit(x)
x['Labels'] = kmean.labels_

In [ ]:
print(x)

In [ ]:
grouped_af = x.groupby(['Labels']).mean().round(1)
grouped_af

pada hasil clustering dapat kita simbulkan bahwa

Cluste 0 adalah minuman dengan nilai kalori, carbo, dan sodium rendah
Cluste 1 adalah minuman dengan nilai kalori, carbo, dan sodium tinggi
Cluste 2 adalah minuman dengan nilai kalori, carbo, dan sodium sedang

#Visualisasi Model

In [ ]:
pal = ["#682F2F","#B9C0C9", "#9F8A78","#F3AB60"]
pl = sns.countplot(x=x["Labels"], palette= pal)
pl.set_title("Distribution Of The Clusters")
plt.show()

Pada data diatas kita melihat bahwa label 0 memiliki jumlah data terbanyak, sehingga menu minuman pada dataset ini ternyata mayoritas rendah kalori, sodium, dan karbohidrat

In [ ]:
pal = ["#682F2F","#B9C0C9", "#9F8A78","#F3AB60"]

pl = sns.scatterplot(x=x["Calories"], y=x["Sodium"],hue=x["Labels"], palette= pal)
pl.set_title("Clustering berdasarkan calories dan sodium")
for label in x['Labels']:
  pl.annotate(label,
               (x[x['Labels']==label]['Calories'].mean(),
                x[x['Labels']==label]['Sodium'].mean()),
                horizontalalignment = 'center',
                verticalalignment = 'center',
                size = 20, weight='bold',
                color='black')
plt.legend()
plt.show()

In [ ]:
pal = ["#682F2F","#B9C0C9", "#9F8A78","#F3AB60"]

pl = sns.scatterplot(x=x["Calories"], y=x["Carb. (g)"],hue=x["Labels"], palette= pal)
pl.set_title("Clustering berdasarkan Calories dan Carb. (g)")
for label in x['Labels']:
  pl.annotate(label,
               (x[x['Labels']==label]['Calories'].mean(),
                x[x['Labels']==label]['Carb. (g)'].mean()),
                horizontalalignment = 'center',
                verticalalignment = 'center',
                size = 20, weight='bold',
                color='black')
plt.legend()
plt.show()

In [ ]:
pal = ["#682F2F","#B9C0C9", "#9F8A78","#F3AB60"]

pl = sns.scatterplot(x=x["Sodium"], y=x["Carb. (g)"],hue=x["Labels"], palette= pal)
pl.set_title("Clustering berdasarkan Sodium dan Carb. (g)")
for label in x['Labels']:
  pl.annotate(label,
               (x[x['Labels']==label]['Sodium'].mean(),
                x[x['Labels']==label]['Carb. (g)'].mean()),
                horizontalalignment = 'center',
                verticalalignment = 'center',
                size = 20, weight='bold',
                color='black')
plt.legend()
plt.show()

#Evaluasi Model

In [ ]:
x_test = df.drop(['Menu','Fat (g)','Fiber (g)','Protein'], axis=1)

In [ ]:
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings("ignore")

# Silhouette analysis
range_n_clusters = [2, 3, 4]

for num_clusters in range_n_clusters:

    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(x_test)

    cluster_labels = kmeans.labels_

    # silhouette score
    silhouette_avg = silhouette_score(x_test, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))

Pada pengujian menggunakan silhouete score, kita melihat bahwa jumlah cluster 3memiliki score terbaik yaitu 0.7246

#Simpan Model

In [ ]:
x.to_excel("output_cluster.xlsx")